In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy.signal import convolve
import sys,os
from burst_utils import find_burst, boxcar_kernel

In [ ]:
#Formatting
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 14}

matplotlib.rc('font', **font)

In [ ]:
def get_mad(ts):
    return np.median(np.abs(ts - np.median(ts)))

def normalise(ts):
    return ts/(1.4826*get_mad(ts))


def Gaussian1D(x,sig,x0):
    return np.exp(-(x-x0)*(x-x0)/(2*sig*sig + MIN_FLOAT))

def linear(x,a,b):
    return a*x + b

def exp_decay(x,tau,x0):
    res = np.zeros(len(x)) + MIN_FLOAT
    #res[x <= x0] = MIN_FLOAT
    res[x > x0] = np.exp(-(x[x>x0]-x0)/(tau+MIN_FLOAT))
    return res

def exp_gauss(x,x0,amp,sig,tau,eps):
    gx0 = np.mean(x)
    g = Gaussian1D(x,sig,gx0)
    ex = exp_decay(x,tau,x0)
    conv = convolve(g,ex,"same")
    conv /= np.max(conv) + MIN_FLOAT
    return amp*conv + eps

def exp_gauss_4(x,x1,amp1,sig1,tau1,
               x2,amp2,sig2,tau2,
               x3,amp3,sig3,tau3,
               x4,amp4,sig4,tau4):
    g1 = exp_gauss(x,x1,amp1,sig1,tau1,0)
    g2 = exp_gauss(x,x2,amp2,sig2,tau2,0)
    g3 = exp_gauss(x,x3,amp3,sig3,tau3,0)
    g4 = exp_gauss(x,x4,amp4,sig4,tau4,0)
    return g1 + g2 + g3 + g4

def lnlike(theta, x, y):
    model = exp_gauss_4(x,*theta)
#    inv_sig = 1./(model**2)
    chisqr = -0.5*(np.sum((y-model)**2))
    return chisqr

In [ ]:
#Load npy
#npy_fil = 'A_117_dm348.8.fits.npy'
npy_fil = 'GBT_B.dm348.8.npy'
#npy_fil = 'C_1164_dm348.8_lores.npy'
#npy_fil = 'D_267_dm348.8_lores.npy'
#npy_fil = 'E_579_dm348.8.fits.npy'
#npy_fil = 'F_639_dm348.8.fits.npy'
#npy_fil = 'G_1549_dm348.8.fits.npy'
#npy_fil = 'GMRT_A.dynamicspec_348.8.npy'
#npy_fil = 'GMRT_B.dynamicspec_349.19.npy'

In [ ]:
subfactor = 1
bandwidth = 400. #MHz
center_frequency = 800. #MHz
file_duration = 83.33 #ms

In [ ]:
def sub_npy(npy_fil, subfactor, file_duration, bandwidth, center_frequency):
    
    npy = np.load(npy_fil)
    npy_sub = np.flipud(np.nanmean(npy.reshape(-1, subfactor, npy.shape[1]), axis=1))
    timeseries = npy_sub.sum(0)

    return npy, npy_sub, timeseries

In [ ]:
npy, npy_sub, timeseries = sub_npy(npy_fil, subfactor, file_duration, bandwidth, center_frequency)
peaks, widths, snrs = find_burst(timeseries)
sampling_time = (file_duration / npy.shape[1])
nchan = npy.shape[0]
freq_res = bandwidth / nchan
print('Sampling Time (ms): ', sampling_time)
print(peaks, widths)